In [2]:
import bs4
import bs4.builder._lxml
import urllib
import pandas as pd
import numpy as np
import requests
import urllib.request, json 
import re
import time
from datetime import date

import hashlib

def hashme(w):
    h = hashlib.md5(w.encode('utf-8'))
    return h.hexdigest()
    
import requests

from urllib.request import Request, urlopen
from urllib.parse import quote

#!pip install PyPDF2
import PyPDF2

from io import StringIO # https://stackoverflow.com/a/18284900
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb') # python three change
    for page in PDFPage.get_pages(infile, pagenums, check_extractable=False):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text
    
def download_pdf(url,path,filename):
    try:        
        filename = path+filename
        f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

        with open(filename, "wb") as code:
            #code.write(f.read())
            code.write(f.content)
        return convert(filename)
    except:
        return "error"

def connect_to_page(url):
    connection = requests.get(url)
    html = connection.text
    soup = bs4.BeautifulSoup(html,"lxml")
    connection.close()
    return soup
def download_pdf_unmasked(url,path,filename):
    
    filename = path+filename
    f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})
    with open(filename, "wb") as code:
        #code.write(f.read())
        code.write(f.content)
    return convert(filename)

    
def download_docx(book_link, book_name):
    the_book = requests.get(book_link, stream=True)
    with open(book_name, 'wb') as f:
      for chunk in the_book.iter_content(1024 * 1024 * 2):  # 2 MB chunks
        f.write(chunk)

In [3]:
BASE = "https://www.dccourts.gov/services/forms"
start = "https://www.dccourts.gov/services/forms?title=&combine=&field_forms_category_value={0}&field_subcategory_select__value=All&field_subcategory_select__value_1=All&sort_by=title&sort_order=ASC&page={1}"
page_nums = [3, 1, 1, 1, 3, 1, 1, 1, 10, 1 ] #number of pages in each cat
soup = connect_to_page(BASE)

cat_wrapper = soup.find("div", class_ = "select-wrapper")

cat_names = []

for i in soup.find_all("option"):
    if i['value'] != "All":
        cat_names.append(i['value'])
    if i['value'] == "Tax":
        break
print(cat_names)

['Civil', 'Criminal', 'Court of Appeals', 'Domestic Violence', 'Family', 'Judge in Chambers', 'Marriage', 'Mediation', 'Probate', 'Tax']


In [8]:
pdfs = {}
today = date.today().strftime("%Y-%m-%d")
files_df = pd.DataFrame([],columns=["id","jurisdiction","source","title","group","url","filename","downloaded", "status"])
form_dest = "All_forms/"
jur = 'DC'



for i in range(len(cat_names)):
    group = cat_names[i]
    print("------------"+group+"------------------")
    for page_num in range(page_nums[i]):
        print(page_num)
        soup = connect_to_page(start.format(cat_names[i], page_num))
        table = soup.find("div", class_ = "table table-bordered table-condensed table-hover table-striped")
        forms = soup.find_all("a", href= True)
        for f in forms:
            if ".pdf" in f['href'] and "http" in f['href']:
                print(f['href'])
                filename = f['href'].split("/")[-1].replace("%20", "_")
                print(filename)
                if "EN" != f.text and "AM" != f.text and "ES" != f.text:
                    title = re.sub(r"\([^()]*\)","",f.text)
                print(title)
                url = f['href']
                if title not in pdfs:
                    pdfs[title] = filename
                    id = hashme(url)
                    status = download_pdf(url, form_dest,id+'.pdf')
                    if status == "error":
                        error = 1
                        print("#####################")
                        print("ERROR")
                        print("#####################")
                    else:
                        error = 0
                    files_df = files_df.append(pd.DataFrame([[id,jur,BASE,title,group,url,filename,today, error]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded", "status"]))

files_df.to_csv("DC_metaData.csv",index=False, encoding="utf-8")

------------Civil------------------
0
https://www.dccourts.gov/sites/default/files/pdf-forms/Forms4a4bAddendum.pdf
Forms4a4bAddendum.pdf
Addendum To L&T Forms 4 & 4
https://www.dccourts.gov/sites/default/files/pdf-forms/Forms4a4bAddendum.pdf
Forms4a4bAddendum.pdf
Addendum To L&T Forms 4 & 4
https://www.dccourts.gov/sites/default/files/2019-09/Affidavit%20in%20Compliance%20with%20LT%20Rule%209%20with%20form%20number%201.pdf
Affidavit_in_Compliance_with_LT_Rule_9_with_form_number_1.pdf
Affidavit in Compliance w/L&T Rule 9 and DCCA Rule 49 
https://www.dccourts.gov/sites/default/files/2019-09/Affidavit%20in%20Compliance%20with%20LT%20Rule%209%20with%20form%20number%201.pdf
Affidavit_in_Compliance_with_LT_Rule_9_with_form_number_1.pdf
Affidavit in Compliance w/L&T Rule 9 and DCCA Rule 49 
https://www.dccourts.gov/sites/default/files/2018-03/Aff-inCompl-w-Sm-Cl-R9-and-DCCA-R49-revMar-2018.pdf
Aff-inCompl-w-Sm-Cl-R9-and-DCCA-R49-revMar-2018.pdf
Affidavit in Compliance w/Small Claims Rule 9 a

In [ ]:
soup = connect_to_page("https://www.dccourts.gov/services/forms?title=&combine=&field_forms_category_value=Civil&field_subcategory_select__value=All&field_subcategory_select__value_1=All&sort_by=title&sort_order=ASC&page=2")
table = soup.find("div", class_ = "table table-bordered table-condensed table-hover table-striped")
forms = soup.find_all("a", href= True)
for f in forms:
        print(f['href'])
        print(f.text)

#main-content

      Skip to main content
    
/coronavirus
See current status of court operations due to Covid
https://www.dccourts.gov/stepstokeepyousafe
steps we've taken to keep you safe
/services/remote-hearing-information
remote hearing information
https://www.dccourts.gov/es/coronavirus
Para español, haga clic aquí
/

#
English
/services/forms?title=&combine=&field_forms_category_value=Civil&field_subcategory_select__value=All&field_subcategory_select__value_1=All&sort_by=title&sort_order=ASC&page=2
English
/am/services/forms?title=&combine=&field_forms_category_value=Civil&field_subcategory_select__value=All&field_subcategory_select__value_1=All&sort_by=title&sort_order=ASC&page=2
Amharic
/zh-TW/services/forms?title=&combine=&field_forms_category_value=Civil&field_subcategory_select__value=All&field_subcategory_select__value_1=All&sort_by=title&sort_order=ASC&page=2
Chinese (Traditional)
/fr/services/forms?title=&combine=&field_forms_category_value=Civil&field_subcategory_selec